# SageMaker Example

## 1. Create your container repository

open aws console and create a repository for your container: https://us-west-2.console.aws.amazon.com/ecr/create-repository?region=us-west-2

for example `236995464743.dkr.ecr.us-west-2.amazonaws.com/sagemaker_byoc_test`

In [ ]:
# login
!aws ecr get-login-password --region us-west-2 | docker login --username AWS --password-stdin 236995464743.dkr.ecr.us-west-2.amazonaws.com

container = "236995464743.dkr.ecr.us-west-2.amazonaws.com/sagemaker_byoc_test:latest"

## 2. Build the container

demo codes are in `app/`
build and push the docker with following commands:

In [ ]:
!cp app/app_async.py app/app.py
!docker build -t sagemaker_byoc_test .
!docker tag sagemaker_byoc_test:latest {container}
!docker push {container}

## 3. Deploy on SageMaker

define the model and deploy on SageMaker


In [ ]:
import boto3
import sagemaker
from sagemaker import Model

In [ ]:
sess = sagemaker.Session()
role = sagemaker.get_execution_role()

In [ ]:
%%bash
# mymodel will auto download to "/opt/ml/model/" in your container
mkdir mymodel
touch mymodel/your_file
tar czvf mymodel.tar.gz mymodel/
rm -rf mymodel

In [ ]:
s3_code_prefix = f"sagemaker_byoc_test/mymodel"
bucket = sess.default_bucket() 
code_artifact = sess.upload_data("mymodel.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {code_artifact}")

In [ ]:
model = Model(
        name="sagemaker-byoc-test-async",
        model_data=code_artifact,
        image_uri=container,
        role=role,
    )

In [ ]:
from sagemaker.async_inference.async_inference_config import AsyncInferenceConfig

async_config = AsyncInferenceConfig(
    output_path=f"s3://{bucket}/async_inference_output",
    max_concurrent_invocations_per_instance=4,
    # Optionally specify Amazon SNS topics
    # notification_config = {
    # "SuccessTopic": "arn:aws:sns:<aws-region>:<account-id>:<topic-name>",
    # "ErrorTopic": "arn:aws:sns:<aws-region>:<account-id>:<topic-name>",
    # }
)

In [ ]:
# 部署模型到endpoint
endpoint_name = sagemaker.utils.name_from_base("sagemaker-byoc-test-async")
print(f"endpoint_name: {endpoint_name}")
predictor = model.deploy(
    initial_instance_count=1,
    async_inference_config=async_config,
    instance_type='ml.t2.medium',
    endpoint_name=endpoint_name,
)

## 4. Test

you can invoke your model with SageMaker SDK

In [ ]:
runtime = boto3.client('runtime.sagemaker')

import json

# 测试 /invocations
payload = {"data": [1, 2, 3, 4, 5]}
 
input_location = sess.upload_string_as_file_body(json.dumps(payload), bucket, "sagemaker_byoc_test_async_inference/example.json")

response = runtime.invoke_endpoint_async(
    EndpointName=endpoint_name,
    InputLocation=input_location,
)

output_location = response["OutputLocation"]
print(f"OutputLocation: {output_location}")

In [ ]:
import urllib, time
from botocore.exceptions import ClientError


def get_output(output_location):
    output_url = urllib.parse.urlparse(output_location)
    bucket = output_url.netloc
    key = output_url.path[1:]
    while True:
        try:
            return sess.read_s3_file(bucket=output_url.netloc, key_prefix=output_url.path[1:])
        except ClientError as e:
            if e.response["Error"]["Code"] == "NoSuchKey":
                print("waiting for output...")
                time.sleep(2)
                continue
            raise


output = get_output(output_location)
print(f"Output: {output}")